In [76]:
from pymongo import MongoClient
import pprint
import numpy as np
import scipy.stats as stats
import scipy

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.style.use('fivethirtyeight')

import requests

from bs4 import BeautifulSoup

import json
import time
import copy
from datetime import datetime

In [77]:
pokemon_df_complete = pd.read_csv('~/DSI_ii/cap_stones/DSI_CS2/data/pokemongo_new.csv')

In [78]:
pokemon_df_complete

,Unnamed: 0,title,score,id,subreddit,url,num_comments,body,created
0,0,What’s the meta right now for the premier league?,1,mfz6c6,pokemongo,https://www.reddit.com/r/pokemongo/comments/mf...,2,Just wondering what most people would consider...,2021-03-30 04:49:12
1,1,A gaze open him in disappointment,4,mfyy6y,pokemongo,https://i.redd.it/i2nuhrod31q61.jpg,8,NaN,2021-03-30 04:39:06
2,2,Niantic AR Glasses are coming soon!,2,mfyvpl,pokemongo,https://twitter.com/johnhanke/status/137662474...,0,NaN,2021-03-30 04:35:53
3,3,The Higher They Fly... research,3,mfytk6,pokemongo,https://www.reddit.com/r/pokemongo/comments/mf...,3,"My brother just finished defeating Giovanni, g...",2021-03-30 04:33:09
4,4,Can I get still get shadow mewtwo if i beat gi...,2,mfyqmj,pokemongo,https://www.reddit.com/r/pokemongo/comments/mf...,9,So i just complete an Inter-egg-sting developm...,2021-03-30 04:29:36
...,...,...,...,...,...,...,...,...,...
978,978,Raid issue,1,m7fsak,pokemongo,https://www.reddit.com/r/pokemongo/comments/m7...,2,Me and my friend were doing a raid and used ou...,2021-03-18 09:20:19
979,979,Referral Rewards,43,m7fn92,pokemongo,https://i.redd.it/73ahpz54ton61.jpg,24,NaN,2021-03-18 09:12:34
980,980,Only took me about 3 years! Just in time for m...,35,m7f7wz,pokemongo,https://www.reddit.com/r/pokemongo/comments/m7...,7,"I'm so happy to finally hit 40, I've been work...",2021-03-18 08:49:35
981,981,How to get multiple of the same task from the ...,0,m7f6he,pokemongo,https://www.reddit.com/r/pokemongo/comments/m7...,7,"So I’m out trying to get the Gible task, I got...",2021-03-18 08:47:25


In [79]:
pokemon_df = pokemon_df_complete[pokemon_df_complete.created <'2021-03-29 10:49:12']

In [80]:
pokemon_df.drop(pokemon_df.columns[0],axis=1)

,title,score,id,subreddit,url,num_comments,body,created
76,Kyogre.. what are you doing?,42,mffwsy,pokemongo,https://i.redd.it/jpgidk26qvp61.jpg,10,NaN,2021-03-29 10:36:08
77,How’s everyone’s shiny Mew quest going?,1,mfflil,pokemongo,https://www.reddit.com/r/pokemongo/comments/mf...,20,All I can say is I have never walked so much i...,2021-03-29 10:17:17
78,"Oh, nvm, there’s another one a block over!",303,mffjio,pokemongo,https://i.redd.it/0zhcza88mvp61.jpg,14,NaN,2021-03-29 10:14:03
79,Quick Rant,4,mffi4a,pokemongo,https://www.reddit.com/r/pokemongo/comments/mf...,3,Anyone else sick of reward pokemon (like for r...,2021-03-29 10:11:53
80,Not shiny rayquaza,3,mffb8v,pokemongo,https://www.reddit.com/r/pokemongo/comments/mf...,1,"It may not be a shiny rayquaza, nor are its iv...",2021-03-29 10:00:49
...,...,...,...,...,...,...,...,...
978,Raid issue,1,m7fsak,pokemongo,https://www.reddit.com/r/pokemongo/comments/m7...,2,Me and my friend were doing a raid and used ou...,2021-03-18 09:20:19
979,Referral Rewards,43,m7fn92,pokemongo,https://i.redd.it/73ahpz54ton61.jpg,24,NaN,2021-03-18 09:12:34
980,Only took me about 3 years! Just in time for m...,35,m7f7wz,pokemongo,https://www.reddit.com/r/pokemongo/comments/m7...,7,"I'm so happy to finally hit 40, I've been work...",2021-03-18 08:49:35
981,How to get multiple of the same task from the ...,0,m7f6he,pokemongo,https://www.reddit.com/r/pokemongo/comments/m7...,7,"So I’m out trying to get the Gible task, I got...",2021-03-18 08:47:25


In [81]:
pokemon_df['low_score'] = pokemon_df['score']<=1

/home/thomas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
pokemon_df

,Unnamed: 0,title,score,id,subreddit,url,num_comments,body,created,low_score
76,76,Kyogre.. what are you doing?,42,mffwsy,pokemongo,https://i.redd.it/jpgidk26qvp61.jpg,10,NaN,2021-03-29 10:36:08,False
77,77,How’s everyone’s shiny Mew quest going?,1,mfflil,pokemongo,https://www.reddit.com/r/pokemongo/comments/mf...,20,All I can say is I have never walked so much i...,2021-03-29 10:17:17,True
78,78,"Oh, nvm, there’s another one a block over!",303,mffjio,pokemongo,https://i.redd.it/0zhcza88mvp61.jpg,14,NaN,2021-03-29 10:14:03,False
79,79,Quick Rant,4,mffi4a,pokemongo,https://www.reddit.com/r/pokemongo/comments/mf...,3,Anyone else sick of reward pokemon (like for r...,2021-03-29 10:11:53,False
80,80,Not shiny rayquaza,3,mffb8v,pokemongo,https://www.reddit.com/r/pokemongo/comments/mf...,1,"It may not be a shiny rayquaza, nor are its iv...",2021-03-29 10:00:49,False
...,...,...,...,...,...,...,...,...,...,...
978,978,Raid issue,1,m7fsak,pokemongo,https://www.reddit.com/r/pokemongo/comments/m7...,2,Me and my friend were doing a raid and used ou...,2021-03-18 09:20:19,True
979,979,Referral Rewards,43,m7fn92,pokemongo,https://i.redd.it/73ahpz54ton61.jpg,24,NaN,2021-03-18 09:12:34,False
980,980,Only took me about 3 years! Just in time for m...,35,m7f7wz,pokemongo,https://www.reddit.com/r/pokemongo/comments/m7...,7,"I'm so happy to finally hit 40, I've been work...",2021-03-18 08:49:35,False
981,981,How to get multiple of the same task from the ...,0,m7f6he,pokemongo,https://www.reddit.com/r/pokemongo/comments/m7...,7,"So I’m out trying to get the Gible task, I got...",2021-03-18 08:47:25,True


In [83]:
from bs4 import BeautifulSoup

In [84]:
import unicodedata


def remove_accents(input_str:str) -> str:
    '''Removes accents from input string'''
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()

def filter_tokens(tokens:list, stops:object) -> list:
    """Filters tokens base on membership in stop list"""
#     split_punc = lambda x: 
    res = []
    check = [".", "-"]
    for token in tokens:
        if token not in stops and token.isalpha():
            if check[0] in token:
                res += token.partition(check[0])
            elif check[1] in token:
                res += token.partition(check[1])
            else:
                res.append(token)
    return res

In [85]:
from collections import Counter
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import json
from pprint import pprint
import string
from sklearn.feature_extraction.text import TfidfVectorizer

# NLTK Modules
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from src.cleaner import clean_data
from src.helpers import *

In [86]:
nlp_df = pokemon_df['body']
nlp_df = nlp_df.fillna('')


In [87]:
import string
import unicodedata

import nltk

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams
from nltk import pos_tag
from nltk import RegexpParser

In [88]:
corpus= nlp_df.to_list()

In [95]:
y = np.array([pokemon_df['low_score']])

In [89]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [60]:
# vectorizer2 = CountVectorizer(ngram_range=(1,2))
# vectorizer2.fit_transform(corpus)
# vectorizer2.get_feature_names()

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [73]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
X.shape

(907, 4526)

In [90]:
from collections import Counter, defaultdict
import numpy as np
import itertools

class BernoulliNaiveBayes(object):
    """
    Attributes
    ----------
    alpha : float
        Laplace smoothing constant.
    Methods
    -------
    fit(X, y)
        Create the lookup dictionaries used to calculate likelihoods
    predict(X)
        Hard-classify data with the fitted Naive Bayes model
    """
    def __init__(self, alpha=1.0):
        """
        Parameters
        ----------
        alpha : float
            Laplace smoothing constant.  Default is 1, which means something
            that had never been seen before would be assigned a probability of
            1/2 of appearing in a document
        """
        # number of documents of each class
        self.class_counts = defaultdict(int)

        # number of documents for each word for each class
        self.class_feature_counts = defaultdict(Counter)

        self.alpha = float(alpha)
        self.p = None
        self.vocab = set()

    def _compute_likelihoods(self, X, y):
        '''Populates class_counts and class_feature_counts
        Parameters
        ----------
        X : Two-dimensional array-like
            Array of input features, with each inner object representing a
            document or observation
        y : Numpy array
            Array of class labels (e.g. spam / not spam)
        '''
        for doc, label in zip(X,y):
            self.class_counts[label] += 1
            for word in set(doc):
                self.class_feature_counts[label][word] +=1

    def fit(self, X, y):
        '''Fits a Naive Bayes model to explain the relationship between X and y
        Parameters
        ----------
        X : Two-dimensional array-like
            Array of input features, with each inner object representing a
            document or observation
        y : Numpy array
            Array of class labels (e.g. spam / not spam)
        '''

        # Compute number of features, aka the number of unique words (vocabulary)
        for document in X:
            for word in document:
                self.vocab.add(word)
        self.p = len(self.vocab)

        # Compute document count for each word for each class
        self._compute_likelihoods(X, y)

    def posteriors(self, X):
        '''Calculates the log-likelihood of each class for each input data point
        Parameters
        ----------
        X : Two-dimensional array-like
            Array of input features, with each inner object representing a
            document or observation
        Returns
        -------
        posteriors : list of dictionaries
            Each dictionary represents the predicted probability of being in a
            particular class, where key=label and value=prediction
        Notes
        -----
        The Naive Bayes class allows targets besides just zero and one, and a
        list of dictionaries enables this functionality
        '''

        #Make sure you implement Laplace smoothing when calculating probabilities!

        #This is what your output should look like - a list of dictionaries
        sample = [{"some_class":.123, "some_other_class":.451},
                  {"some_class":.888, "some_other_class":.177}]
        total_docs = sum(self.class_counts.values())
        posts = []
        for doc in X:
            post = {}
            for label, count in self.class_counts.items():
                log_prob = np.log(count/total_docs)
                doc_words = set(doc)
                for word in self.vocab:
                    p_num = self.alpha + self.class_feature_counts[label][word]
                    p_denom = 2*self.alpha + self.class_counts[label]
                    p = p_num/p_denom
                    if word in doc_words:
                        log_prob += np.log(p)
                    else:
                        log_prob += np.log(1-p)
                post[label] = log_prob
            posts.append(post)
        return posts

    def predict(self, X):
        """Uses a fitted Naive Bayes model to predict class labels
        Parameters
        ----------
        X : Two-dimensional array-like
            Array of input features, with each inner object representing a
            document or observation
        Returns
        -------
        predictions : Numpy array
            Vector of class predictions for provided data X
        """
        predictions = []

        #Iterate through each dictionary
        for post in self.posteriors(X):
            #Find the label that corresponds to the highest probability
            pred = max(post.keys(), key=(lambda label: post[label]))
            predictions.append(pred)
        return np.array(predictions)

    def score(self, X, y):
        """Calculates the accuracy, the percentage of correct predictions
        Parameters
        ----------
        X : Two-dimensional array-like
            Array of input features, with each inner object representing a
            document or observation
        Returns
        -------
        accuracy : float
            A number representing the classification accuracy on data X
        """
        return np.mean(self.predict(X) == y)

In [100]:
X.shape

(907, 4526)

In [102]:
import numpy as np
from pymongo import MongoClient
from nltk.tokenize import word_tokenize, wordpunct_tokenize, RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB as SKMultinomialNB

In [106]:
y=y.T

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [110]:
nbBern = BernoulliNaiveBayes()
nbBern.fit(X,y)

TypeError: unhashable type: 'csr_matrix'